In [1]:
#OCO2

import h5py
import os
import datetime
import geopandas as geo
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.path as mplPath
import time

%matplotlib inline
plt.style.use('classic')

x = [[],[],[],[],[],[]]

filedir = "C:/Users/asus/Desktop/02_oco_2_2020_working/2020"

for filename in os.listdir(filedir):
    if filename.endswith(".hdf"):
        
        infile      = h5py.File(os.path.join(filedir,filename),"r")
        print(os.path.join(filedir,filename))
        time_string = infile["time"][0:] 
        latitude    = infile["latitude"][0:]
        longitude   = infile["longitude"][0:]
        xco2        = infile["xco2"][0:]
        xco2U       = infile["xco2_uncertainty"][0:]
        xfla        = infile["xco2_quality_flag"][0:]
 
        infile.close()
    
    x[0].extend(latitude)
    x[1].extend(longitude)
    x[2].extend(xco2)
    x[3].extend(time_string)
    x[4].extend(xco2U)
    x[5].extend(xfla)
    
data = np.zeros((len(x[0])),dtype=[('lat','f8'),('lon','f8'),('co2','f8'),('time','U10'),('var','f8'),
                                   ('flag','i4')])
data['lat'] = x[0] 
data['lon'] = x[1]
data['co2'] = x[2] 
data['time']= x[3] 
data['var'] = x[4]
data['flag']= x[5]


a      = np.loadtxt('./lat_long.txt')
bbPath = mplPath.Path(a[:,1:])

x             = data['time'][(data['lat']<44)&(data['lat']>11)&(data['lon']<65)&(data['lon']>23)
                             &(data['flag']==0)].size
data_n        = np.zeros(x,dtype=[('lat','f8'),('lon','f8'),('co2','f8'),('time','U10'),('var','f8')])
data_n['lat'] = data['lat'][(data['lat']<44)&(data['lat']>11)&(data['lon']<65)&(data['lon']>23)&(data['flag']==0)]
data_n['lon'] = data['lon'][(data['lat']<44)&(data['lat']>11)&(data['lon']<65)&(data['lon']>23)&(data['flag']==0)]
data_n['co2'] = data['co2'][(data['lat']<44)&(data['lat']>11)&(data['lon']<65)&(data['lon']>23)&(data['flag']==0)]
data_n['time']= data['time'][(data['lat']<44)&(data['lat']>11)&(data['lon']<65)&(data['lon']>23)&(data['flag']==0)]
data_n['var'] = data['var'][(data['lat']<44)&(data['lat']>11)&(data['lon']<65)&(data['lon']>23)&(data['flag']==0)]

k   = 1
f   = 0
lat = []
lon = []
tim = []
co2 = []
var = []

for i,j in enumerate(data_n['time']):
    l                 = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(int(j)))
    if float(l.split('-')[1]) == f:
        data_n['time'][i] = data_n['time'][i-1]
    else:
        f = float(l.split('-')[1])
        data_n['time'][i] = k
        k = k+1
    con = bbPath.contains_point(np.array([data_n['lat'][i],data_n['lon'][i]]))
    if con:
        lat.append(np.array([data_n['lat'][i],data_n['lon'][i]])[0])
        lon.append(np.array([data_n['lat'][i],data_n['lon'][i]])[1])
        tim.append(data_n['time'][i])
        co2.append(data_n['co2'][i])
        var.append(data_n['var'][i])

data_3n = np.zeros(len(lat),dtype=[('lat','f8'),('lon','f8'),('time','i4'),('co2','f8'),('var','f8')])
data_3n['lat'] = lat
data_3n['lon'] = lon
data_3n['time']= tim
data_3n['co2'] = co2
data_3n['var'] = var

#insert the destination address: 
np.savetxt('./oco-2_middle_east_2020.txt',data_3n,fmt='     '.join(['%1.4f']*2+['%i']+['%1.4f']+['%1.4f']),
           header= 'latitude  longitud    time      co2    variance')

ModuleNotFoundError: No module named 'geopandas'